In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [28]:
train_data = pd.read_csv('dataset/mnist/train.csv')
test_data = pd.read_csv('dataset/mnist/test.csv')

train_data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
train = np.array(train_data).T
test = np.array(test_data).T

train

array([[1, 0, 1, ..., 7, 6, 9],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [32]:
X_train = train[1:]
y_train = train[0]

X_test = test[1:]
y_test = test[0]

print(X_train.shape)
print(y_train.shape)

(784, 42000)
(42000,)


In [33]:
y_train 

array([1, 0, 1, ..., 7, 6, 9])

In [44]:
activation_functions = {
  'sigmoid': lambda x: 1 / (1 + np.exp(-x)),
  'tanh': lambda x: np.tanh(x),
  'relu': lambda x: np.maximum(0, x),
  'softmax': lambda x: np.exp(x) / np.sum(np.exp(x), axis=1, keepdims=True)
}

derivative_activation_functions = {
  'sigmoid': lambda x: x * (1 - x),
  'tanh': lambda x: 1 - x ** 2,
  'relu': lambda x: (x > 0).astype(float),
  'softmax': lambda x: x * (1 - x)
}

In [56]:
class Layer:
  def __init__(self, n_inputs: float, n_neurons: float, activation):
    self.weights = np.random.uniform(-0.5, 0.5, (n_inputs, n_neurons))
    self.biases = np.zeros((1, n_neurons))
    self.activation = activation
    self.output = None
    self.activation_output = None

  def forward(self, inputs): 
    self.output = np.dot(inputs, self.weights) + self.biases
    self.activation_output = self.activation(self.output)
    return self.activation_output

In [ ]:
class MLP:
  def __init__(self, input_layer: int, hidden_layers: int, output_layer: int, learning_rate: float = 0.001, epochs: int = 500, activation: str = 'relu'):
    self.input_layer = input_layer                    
    self.hidden_layer = hidden_layer
    self.output_layer = output_layer
    self.learning_rate = learning_rate
    self.epochs = epochs                                 
    self.activation = activation_functions[activation]
    self.deriv = derivative_activation_functions[activation]

    self.layers = [Layer(input_layer, hidden_layer, activation), Layer(hidden_layer, output_layer, activation_functions['softmax'])]

  def fit(self, X, y):
    for epoch in range(self.epochs):
      for layer in self.layers:
        layer.forward(X[i])
      if epoch % 100 == 0:
        print(f'Epoch: {epoch}, Error: {self.error(y)}')